## 2.3.통계 기반 기법
### 2.3.1. 파이썬으로 말뭉치 전처리하기
- 텍스트 데이터를 단어로 분하랗고 그 분할된 단어들을 단어 ID 목록으로 변환


In [30]:
# example
text = 'You say goodbye and I say hello.'

In [5]:
text1 = text.lower() # 모든 문자를 소문자로 변환
text1 = text1.replace('.',' .') # split(' ')로 공백을 기준으로 모든 단어를 분할하기 위함
text1

'you say goodbye and i say hello .'

In [6]:
words = text1.split(' ') # 공백을 기준으로 모든 단어를 분할
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [69]:
# # 정규표현식 이용해서 단어 단위로 분할하기
# import re
# text2 = re.split('(\W)',text)
# text2

['You',
 ' ',
 'say',
 ' ',
 'goodbye',
 ' ',
 'and',
 ' ',
 'I',
 ' ',
 'say',
 ' ',
 'hello',
 '.',
 '']

- 단어를 텍스트 그대로 조작하는 것은 여러 면에서 불편
- 그래서 단어에 ID를 부여하고, ID의 리스트로 이용할 수 있도록 한 번 더 손질한다.
- 이를 위한 사전 준비로 python의 딕셔너리를 이용해 단어 ID와 단어를 짝지어주는 대응표를 작성한다.

In [13]:
word_to_id = {} # 단어(key)에서 단어 ID(key)로의 변환 
id_to_word = {} # 단어 ID(key)에서 단어(value)로의 변환

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word


In [14]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [15]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [16]:
id_to_word[1]

'say'

In [17]:
word_to_id['hello']

5

- `단어 목록`을 `단어 ID 목록`으로 변경해보기

In [18]:
import numpy as np

corpus = [word_to_id[w] for w in words]
corpus - np.array(corpus)
corpus

[0, 1, 2, 3, 4, 1, 5, 6]

- 위에서 한 단계씩 진행한 처리를 한 데 모아 `preprocess()` 함수 만들기

In [31]:
text

'You say goodbye and I say hello.'

In [32]:
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[word] for word in words])

    return corpus, word_to_id, id_to_word

In [33]:
# preprocess() 함수 활용해서 말뭉치 전처리하기

text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)

### 2.3.2 단어의 분산 표현
- 단어의 분산 표현은 단어의 의미를 정확하게 파악할 수 있는 벡터 표현
- 단어의 분산 표현은 단어를 고정 길이의 밀집 벡터(dense vector)로 표현.
    - 밀집 벡터 : 대부분의 원소가 0이 아닌 실수인 벡터
    

### 2.3.3. 분포 가설
- '단어의 의미는 주변 단어에 의해 형성된다.'는 가설
- 단어 자체에는 의미가 없고, 그 단어가 사용된 `맥락(context)`이 의미를 형성한다.
![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-3.png?raw=true)
- 맥락은 특정 단어를 중심에 둔 그 주변 단어를 으미ㅣ
- 맥락의 크기(주변 단어를 몇 개나 볼 것인가) = `윈도우 크기(window size)`
    - 상황에 따라 왼쪽 단어만 혹은 오른쪽 단어만 사용하기도 하며, 문장의 시작과 끝을 고려할 수 있다.
    

### 2.3.4. 동시 발생 행렬
- 분포 가설에 기초해 단어를 베거로 나타내는 방법
- 주변 단어를 `세어 보는` 방법: 즉, 어떤 단어에 주목했을 때, 그 주변에 어떤 단어가 몇 번이나 등장하는지 세어 집계하는 방법. 
    - 이를 `통계 기반 기법` 이라고 한다.

In [34]:
import sys
sys.path.append('..')
import numpy as np 
# from common.util import preprocess => 위에 정의한 preprocess 함수(method)

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print(id_to_word)



[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


- 단어 "you"의 맥락 세어보기
    ![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-4.png?raw=true)

    - 단어 "you"의 맥락은 "say"라는 단어 하나뿐이고, 이를 표로 정리하면 아래와 같다.
    ![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-5.png?raw=true)
        - 이 표를 바탕으로 "you"라는 단어를 `[0, 1, 0, 0, 0, 0, 0]`라는 벡터로 표현할 수 있다.

- 이 작업을 모든 단어(총 7개의 단어)에 대해 수행해보면, 아래와 같다.
- 이는 모든 단어에 대해 동시 발생하는 단어를 표에 정리한 것. (`동시 발생 행렬, co-occurrence matrix`라고 한다.)
    - 각 행은 해당 단어를 표현한 베겉가 된다. 
    ![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/fig%202-7.png?raw=true)

#### 동시 발생 행렬 파이썬으로 구현해보기
- 그림 그대로 손으로 구현해보기

In [43]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0]
], dtype=np.int32)


In [40]:
print(C[0]) # ID가 0인 단어의 벡터 표현

[0 1 0 0 0 0 0]


In [41]:
print(C[4]) # ID가 4인 단어의 벡터 표현

[0 1 0 1 0 0 0]


In [42]:
print(C[word_to_id['goodbye']]) # "goodbye"의 벡터 표현

[0 1 0 1 0 0 0]


- 동시발생 행렬을 만들어주는 함수 구현하기

In [44]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    """
    args
    - corpus: 단어 ID의 리스트
    - vocab_size: 어휘 수
    - window_size: 윈도우 크기
    """
    corpus_size = len(corpus)
    co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size +1):
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    
    return co_matrix



In [50]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(set(corpus)) # vocab_size: 중복되지 않는 unique한 어휘 수
print(vocab_size)
print(corpus) # corpus: 주어진 text에 있는 모든 어휘 수 (중복 있음)
create_co_matrix(corpus, vocab_size, window_size=1)

7
[0 1 2 3 4 1 5 6]


array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

In [51]:
create_co_matrix(corpus, vocab_size, window_size=1) == C

array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True]])

### 2.3.5. 벡터간 유사도
- 벡터 내적, 유클리드 거리, 코사인 유사도 등 방법이 다양하지만, 단어 벡터 유사도를 나타낼 때 코사인 유사도를 자주 이용한다.
- 두 벡터 $x = (x_1, x_2, x_3, ..., x_n)과 y = (y_1, y_2, y_3, ..., y_n)$이 있다면, 코사인 유사도는 아래와 같은 식으로 정의된다.
    ![](https://github.com/yesinkim/Deep-Learning-From-Scratch2/blob/main/deep_learning_2_images/e%202-1.png?raw=true)

- 코사인 유사도 식의 분자에는 벡터의 내적이, 분모에는 각 벡터의 노름(norm)이 등장한다.
    - `노름(norm)`: 벡터의 크기를 나타낸 것으로, 여기에서는 **L2 노름**을 계산한다.
    - `L2 노름`: 벡터의 각 원소를 제곱해 더한 후 다시 제곱근을 구해 계산
- 코사인 유사도 식의 핵심은 **벡터를 정규화하고 내적을 구하는 것**이다.
- 코사인 유사도는 **두 벡터가 가리키는 방향이 얼마나 비슷한가**를 나타낸다.
    - 두 벡터의 방향이 완전히 같다면 코사인 유사도가 1, 완전히 반대라면 -1

In [55]:
# 이렇게만 코사인 유사도를 정의한다면, 문제가 하나 발생한다. 
# 인수로 제로 벡터(원소가 모두 0인 벡터)가 들어오면, `0으로 나누기` 오류가 발생한다. => 해결법: 나눌 때 분모에 작은 값 더해주기 (eps(앱실론), 1e-8로 설정)

def cos_similarity(x: np.ndarray, y: np.ndarray):
    nx = x / np.sqrt(np.sum(x**2)) # x의 정규화
    ny = y / np.sqrt(np.sum(y**2)) # y의 정규화
    return np.dot(nx, ny)

- 작은 값으로 eps를 `1e-8`로 정해 사용했는데, 이정도 작은 값이라면 일반적으로 부동소수점 계산 시, **반올림**되어 다른 값에 **흡수**된다.
- 아래 구현에서는 해당 값이 노름에 흡수되기 때문에 대부분의 경우, eps를 더한다고 해서 최종 계산 결과에는 영향을 주지 않는다.
- 그러나 벡터의 노름이 0인 경우, 작은 값이 그대로 유지되어 0으로 나누기 오류가 나는 것을 방지한다.

In [56]:
# eps 더해줘서 오류 수정
def cos_similarity(x: np.ndarray, y: np.ndarray, eps=1e-8):
    nx = x / np.sqrt(np.sum(x**2) + eps) # x의 정규화
    ny = y / np.sqrt(np.sum(y**2) + eps) # y의 정규화
    return np.dot(nx, ny)

In [57]:
import sys
sys.path.append('.')
# from common.util import preprocess, create_co_matrix, cos_similarity # 모두 위에 구현해놓은 함수로 대체 

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']] # "you"의 단어 임베딩
c1 = C[word_to_id['i']]  # "i"의 단어 임베딩

print(cos_similarity(c0, c1)) # 코사인 유사도 값은 -1 ~ 1 사이의 값으로 나온다.

0.7071067758832467


### 2.3.6. 유사 단어 랭킹 표시

In [78]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    """
    args
    - query: 검색어(단어)
    - word_to_id: 단어에서 단어 ID로의 딕셔너리
    - id_to_word: 단어 ID에서 단어로의 딕셔너리
    - word_matrix: 단어 벡터들을 한 데 모은 행렬. 각 행에는 대응하는 단어의 벡터가 저장되어 있다고 가정.
    - top: 상위 몇 개까지 출력할지 결정
    """
    # 1. 검색어를 꺼낸다.
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return
    
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 2. 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 3. 코사인 유사도를 기준으로 내림차순 출력
    count = 0
    for i in (-1 * similarity).argsort(): # 오름차순 기준으로 유사도 높은 것부터 for문으로 뽑아온다.
        if id_to_word[i] == query: # 자기 자신에 대한 유사도 제외
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return



In [79]:
# numpy argsort: 넘파이 배열의 원소를 "오름차순"으로 정렬. 반환값은 배열의 인덱스 값.
x = np.array([100, -20, 2])
x.argsort()

# 따라서 유사도가 "큰" 값으로 정렬하기 위해서는 => 넘파이 배열의 각 원소에 마이너를 곱한 후, argsort() 메소드를 호출하면 된다.
(-x).argsort()

array([0, 2, 1])

In [80]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067758832467
 i: 0.7071067758832467
 hello: 0.7071067758832467
 say: 0.0
 and: 0.0


- "you"와 가장 유사한 단어들: "goodbye", "i", "hello"
    - "i"가 유사한 단어로 뽑힌 것은 직관적으로 이해가 되는 부분
    - 그러나 "goodbye"와 "hello"가 "you"와 코사인 유사도가 높다는 것은 직관적으로 이해가 잘 안됨
        - 왜 그런 결과가 나왔나?: 말뭉치의 크기가 너무 작기 때문.